# Giriş işlemleri

In [ ]:
%%capture
!pip install -q unsloth transformers accelerate bitsandbytes

## Kütüphane içe aktarma

In [ ]:
import os
import pandas as pd
from unsloth import FastLanguageModel
import torch
from trl import GRPOTrainer, GRPOConfig
from datasets import Dataset
from typing import List, Optional, Callable, Any, Dict, Union

In [ ]:
def is_colab() -> bool:
    """
    Google Colab ortamında çalışıp çalışmadığını kontrol eden fonksiyon.

    Args:
        None

    Returns:
        bool: Eğer kod Google Colab'da çalışıyorsa True, aksi halde False döndürür
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    """
    Eğer kod Google Colab ortamında çalışıyorsa, Google Drive'ı bağlar ve
    kök dizini Google Drive içindeki "turkish_gpt2_finetuning" klasörü olarak ayarlar.
    """
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive'ı Colab ortamına bağlar
    kok_dizin = "/content/drive/MyDrive/turkish_gpt2_finetuning"  # Drive içindeki çalışma klasörünü belirler
else:
    """
    Eğer kod yerel bir ortamda çalışıyorsa, kök dizini mevcut çalışma dizini olarak ayarlar.
    """
    kok_dizin = os.getcwd()  # Mevcut çalışma dizinini alır

# Belirlenen kök dizini kullanıcıya bilgi olarak gösterir
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

## Veri kümesi biçimlendirme, yol ve model yolu tanımlamaları

In [ ]:
# Veri kümesi yolu ve karıştırma seed değeri tanımlamaları
veri_kumesi_yolu = os.path.join(kok_dizin, "soru_cevap.csv")
veri_kumesi_karistirma = 571  # Karıştırmada kullanılacak sabit seed değeri

# Model kaydetme dizinleri
# gpt4o verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_medium_gpt4o")
gpt2_large_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_large_gpt4o")

# deepseek verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_medium_deepseek")
gpt2_large_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_large_deepseek")

# Model adı tanımlamaları
gpt2_medium_model_adi = "ytu-ce-cosmos/turkish-gpt2-medium"
gpt2_large_model_adi = "ytu-ce-cosmos/turkish-gpt2-large"

# Veri kümesi sütun tanımlamaları
soru_sutunu = "Soru"
gpt4o_cevap_sutunu = "gpt4o cevabı"
deepseek_cevap_sutunu = "deepseek cevabı"

# Özel token tanımlamaları (eğitim formatı için)
soru_baslangic = "<SORU>"
soru_bitis = "</SORU>"
cevap_baslangic = "<CEVAP>"
cevap_bitis = "</CEVAP>"
ornek_bitis = "<|endoftext|>"  # GPT-2'nin EOS tokeni

# Tokenizer'a eklenecek özel tokenler listesi
ozel_tokenler = [soru_baslangic, soru_bitis, cevap_baslangic, cevap_bitis]

## Veri kümesi okuma

In [ ]:
# CSV dosyasını okuma
try:
    # CSV dosyasını okuyoruz
    df = pd.read_csv(veri_kumesi_yolu)

    # İstenen sütunların varlığını kontrol ediyoruz
    gereken_sutunlar = ["Soru", gpt4o_cevap_sutunu, deepseek_cevap_sutunu]

    for sutun in gereken_sutunlar:
        if sutun not in df.columns:
            print(f"'{sutun}' sütunu veri kümesinde bulunamadı!")

    # İstenen sütunları ayıklama
    soru_cevap_df = df[gereken_sutunlar]

    # Doğrulama için ilk birkaç satırı görüntüleme
    print("Veri kümesinin ilk birkaç satırı:")
    print(soru_cevap_df.head(5))

    print(f"\nToplam {len(soru_cevap_df)} soru-cevap çifti bulundu.")

except FileNotFoundError:
    print(f"'{veri_kumesi_yolu}' dosyası bulunamadı. Lütfen dosya yolunu kontrol ediniz.")
except Exception as e:
    print(f"Veri okuma hatası: {e}")

In [ ]:
# Veri kümesini belirtilen seed değeriyle karıştırma
shuffled_df = soru_cevap_df.sample(frac=1, random_state=veri_kumesi_karistirma)

# İndeksleri sıfırlama
shuffled_df = shuffled_df.reset_index(drop=True)

# Karıştırılmış veri kümesinin ilk birkaç satırını gösterme
print("Karıştırılmış veri kümesinin ilk birkaç satırı:")
print(shuffled_df.head(5))

print(f"\nKarıştırılmış veri kümesi boyutu: {len(shuffled_df)} satır")

# Veri kümesi işleme

## Veri kümesi biçimlendirme fonksiyonu

In [ ]:
def veri_kumesini_egitim_formatina_donustur(df: pd.DataFrame, cevap_sutunu: str,
                                            soru_baslangic: str = soru_baslangic,
                                            soru_bitis: str = soru_bitis,
                                            cevap_baslangic: str = cevap_baslangic,
                                            cevap_bitis: str = cevap_bitis,
                                            ornek_bitis: str = ornek_bitis) -> list:
    """
    Veri kümesindeki soru ve cevapları GPT-2 eğitimi için uygun formata dönüştürür.
    Metinsel formatta örnekler döndürür, tokenize işlemi uygulamaz.

    Args:
        df: Soru ve cevapları içeren DataFrame
        cevap_sutunu: Cevap metinlerini içeren sütunun adı
        soru_baslangic: Soru başlangıç etiketi
        soru_bitis: Soru bitiş etiketi
        cevap_baslangic: Cevap başlangıç etiketi
        cevap_bitis: Cevap bitiş etiketi
        ornek_bitis: Her örneğin sonunu belirten etiket

    Returns:
        list: Eğitim için hazırlanmış metinsel örnekler listesi
    """

    egitim_metinleri = []

    # Her bir soru-cevap çifti için formatlı metinler oluştur
    for _, satir in df.iterrows():
        soru = satir["Soru"].strip()
        cevap = satir[cevap_sutunu].strip()

        # Soru ve cevabı belirli bir formatta birleştir
        bicimlendirilmis_metin = f"{soru_baslangic} {soru} {soru_bitis} {cevap_baslangic} {cevap} {cevap_bitis}{ornek_bitis}"

        egitim_metinleri.append(bicimlendirilmis_metin)

    print(f"Toplam {len(egitim_metinleri)} adet eğitim örneği oluşturuldu.")
    print(f"Örnek biçimi: {soru_baslangic} [Soru] {soru_bitis} {cevap_baslangic} [Cevap] {cevap_bitis}{ornek_bitis}")
    return egitim_metinleri

## Veri kümelerini işle ve oluştur

In [ ]:
veri_kumesi_gpt4o = veri_kumesini_egitim_formatina_donustur(shuffled_df, gpt4o_cevap_sutunu)
veri_kumesi_deepseek = veri_kumesini_egitim_formatina_donustur(shuffled_df, deepseek_cevap_sutunu)

# Eğitim işlemleri

## Model Yükleme ve Eğitme Fonksiyonları

### Eğitim argümanları oluşturma fonksiyonu

In [ ]:
# Eğitim parametreleri
max_seq_length = 1024  # Uzun metinler için artırılabilir
max_prompt_length = 256  # Soru için maksimum uzunluk

def training_arguments_getir(
    kaydetme_dizin: str,
    learning_rate: float = 5e-6,
    batch_size: int = 1,
    grad_accum_steps: int = 16,
    num_gens: int = 6,
    max_steps: int = 250,
    epochs: int = None
) -> GRPOConfig:
    """
    GRPO (Generative Reinforced Policy Optimization) eğitim konfigürasyonu oluşturur.

    Args:
        kaydetme_dizin: Model ve checkpoint'lerin kaydedileceği dizin
        learning_rate: Öğrenme oranı
        batch_size: Cihaz başına eğitim batch büyüklüğü
        grad_accum_steps: Gradyan biriktirme adımları
        num_gens: Bellek kullanımını etkileyen üretim sayısı
        max_steps: Maksimum eğitim adım sayısı
        epochs: Eğitim döngü (epoch) sayısı, None ise max_steps kullanılır

    Returns:
        GRPOConfig: Eğitim için yapılandırma nesnesi
    """
    # GRPO eğitim konfigürasyonu
    training_args = GRPOConfig(
        learning_rate=learning_rate,
        adam_beta1=0.9,
        adam_beta2=0.99,
        weight_decay=0.1,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        optim="paged_adamw_8bit",
        logging_steps=1,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum_steps,
        num_generations=num_gens,
        max_prompt_length=max_prompt_length,
        max_completion_length=max_seq_length - max_prompt_length,
        num_train_epochs=epochs,  # None ise max_steps kullanılır
        max_steps=max_steps if epochs is None else -1,
        save_steps=max_steps // 2,  # Daha sık kaydetmek için ayarlanabilir
        max_grad_norm=0.1,
        report_to="none",  # Metrics raporlama için "wandb" kullanılabilir
        output_dir=kaydetme_dizin,
    )

    return training_args  # Oluşturulan konfigürasyon nesnesini döndür

### Model Yükleme fonksiyonu

In [ ]:
def model_ve_tokenizer_yukle(
    model_adi: str,
    max_seq_length: int = max_seq_length,
    lora_rank: int = 32,
    random_state: int = 571,  # veri_kumesi_karistirma değişkeni burada tanımlı olmayabilir
    target_modules: list = ["c_attn", "c_proj", "c_fc"],
    ozel_tokenler: list = ozel_tokenler,
):
    """
    Unsloth kullanarak bir dil modelini yükler ve LoRA için hazırlar.

    Args:
        model_adi: Kullanılacak modelin adı (örn. "ytu-ce-cosmos/turkish-gpt2-large")
        max_seq_length: Maximum sekans uzunluğu
        lora_rank: LoRA rank değeri
        random_state: Rastgele seed değeri
        target_modules: LoRA için hedeflenecek modül katmanları

    Returns:
        tuple: (model, tokenizer) çifti
    """
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_adi,
        max_seq_length = max_seq_length,
        load_in_4bit = False,  # 16-bit için False
        fast_inference = True,
        max_lora_rank = lora_rank,
        gpu_memory_utilization = 0.6,
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r = lora_rank,
        target_modules = target_modules,
        lora_alpha = lora_rank,
        use_gradient_checkpointing = "unsloth",
        random_state = random_state,
    )
    tokenizer.add_tokens(ozel_tokenler)
    model.resize_token_embeddings(len(tokenizer))
    print(f"{model_adi} modeli Unsloth ile başarıyla yüklendi ve LoRA için hazırlandı.")
    return model, tokenizer

### Eğitici Getirme fonksiyonu

In [ ]:
def trainer_getir(
    model,
    tokenizer,
    training_args: Optional[GRPOConfig],
    veri_kumesi: list,
    egitim_batch_boyutu: int = 1
) -> GRPOTrainer:
    """
    Model, tokenizer ve eğitim argümanlarını kullanarak GRPOTrainer oluşturur.

    Args:
        model: Eğitilecek dil modeli
        tokenizer: Modelle kullanılacak tokenizer
        training_args: GRPO eğitim konfigürasyonu, None ise varsayılan oluşturulur
        veri_kumesi: Eğitim için hazırlanmış metinsel örnekler listesi
        egitim_batch_boyutu: Eğitim için batch boyutu

    Returns:
        GRPOTrainer: Eğitimi yürütecek trainer nesnesi
    """
    # Eğer training_args belirtilmemişse, varsayılan değerlerle oluştur
    if training_args is None:
        training_args = training_arguments_getir(
            kaydetme_dizin="./model_cikti",
            batch_size=egitim_batch_boyutu
        )

    # Metin verilerini tokenize et ve eğitim için hazırla
    def tokenize_function(examples):
        # Tek bir metin listesi olduğu için özel bir işleme
        tokenized = tokenizer(
            examples,
            padding="max_length",
            truncation=True,
            max_length=training_args.max_prompt_length + training_args.max_completion_length,
            return_tensors="pt"
        )
        return tokenized

    # Metinsel verileri tokenize et
    tokenized_dataset = tokenize_function(veri_kumesi)

    # Dataset oluşturma
    from torch.utils.data import Dataset

    class TextDataset(Dataset):
        def __init__(self, tokenized_texts):
            self.input_ids = tokenized_texts["input_ids"]
            self.attention_mask = tokenized_texts["attention_mask"]

        def __len__(self):
            return len(self.input_ids)

        def __getitem__(self, idx):
            # GRPO için input_ids ve attention_mask
            sample = {
                "input_ids": self.input_ids[idx],
                "attention_mask": self.attention_mask[idx],
            }
            return sample

    # Dataset oluştur
    train_dataset = TextDataset(tokenized_dataset)

    print(f"Eğitim için {len(train_dataset)} örnek hazırlandı.")

    # GRPOTrainer oluştur
    trainer = GRPOTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train_dataset,
    )

    return trainer

### Veri kümesi özelinde model eğitme fonksiyonu

In [ ]:
def model_egit_ve_kaydet(
    model,
    tokenizer,
    veri_kumesi,
    model_kaydetme_dizini,
):
    """
    Modeli belirtilen veri kümesi üzerinde eğitir ve kaydeder.

    Args:
        model: Eğitilecek model
        tokenizer: Modelin tokenizer'ı
        veri_kumesi: Eğitim veri kümesi listesi
        model_kaydetme_dizini: Eğitilmiş modelin kaydedileceği dizin

    Returns:
        Eğitilmiş model
    """
    # Eğitim klasörünü oluştur
    os.makedirs(model_kaydetme_dizini, exist_ok=True)

    # Eğitim argümanlarını oluştur
    training_args = training_arguments_getir(
        kaydetme_dizin=model_kaydetme_dizini,
    )

    # Trainer oluştur
    trainer = trainer_getir(
        model=model,
        tokenizer=tokenizer,
        training_args=training_args,
        veri_kumesi=veri_kumesi,
    )

    # Eğitimi başlat
    print(f"Model eğitimine başlanıyor: {model_kaydetme_dizini}")
    trainer.train()

    # Modelin tam versiyonunu kaydet
    model.save_pretrained(model_kaydetme_dizini)
    tokenizer.save_pretrained(model_kaydetme_dizini)
    print(f"Model başarıyla kaydedildi: {model_kaydetme_dizini}")

    del model
    torch.cuda.empty_cache()

## GPT-2 Medium modelini GPT-4o veri kümesi ile eğit

In [ ]:
# GPT-4o veri kümesiyle Medium modeli eğit
print("\nGPT-2 Medium modeli GPT-4o veri kümesi ile eğitiliyor...")
model_medium, tokenizer_medium = model_ve_tokenizer_yukle(gpt2_medium_model_adi)
model_egit_ve_kaydet(
    model_medium,
    tokenizer_medium,
    veri_kumesi_gpt4o,
    gpt2_medium_kaydetme_dizini_gpt4o,
)

## GPT-2 Medium modelini DeepSeek veri kümesi ile eğit

In [ ]:
# DeepSeek veri kümesiyle Medium modeli eğit
print("\nGPT-2 Medium modeli DeepSeek veri kümesi ile eğitiliyor...")
model_medium, tokenizer_medium = model_ve_tokenizer_yukle(gpt2_medium_model_adi)
model_egit_ve_kaydet(
    model_medium,
    tokenizer_medium,
    veri_kumesi_deepseek,
    gpt2_medium_kaydetme_dizini_deepseek,
)

## GPT-2 Large modelini GPT-4o veri kümesi ile eğit

In [ ]:
# GPT-4o veri kümesiyle Large modeli eğit
print("\nGPT-2 Large modeli GPT-4o veri kümesi ile eğitiliyor...")
model_large, tokenizer_large = model_ve_tokenizer_yukle(gpt2_large_model_adi)
model_egit_ve_kaydet(
    model_large,
    tokenizer_large,
    veri_kumesi_gpt4o,
    gpt2_large_kaydetme_dizini_gpt4o,
)

## GPT-2 Large modelini DeepSeek veri kümesi ile eğit

In [ ]:
# DeepSeek veri kümesiyle Large modeli eğit
print("\nGPT-2 Large modeli DeepSeek veri kümesi ile eğitiliyor...")
model_large, tokenizer_large = model_ve_tokenizer_yukle(gpt2_large_model_adi)
model_egit_ve_kaydet(
    model_large,
    tokenizer_large,
    veri_kumesi_deepseek,
    gpt2_large_kaydetme_dizini_deepseek,
)